## 클로저 이미징 소개
EHT는 지금까지 만들어진 망원경 중 가장 해상도가 높은 망원경입니다. 이 망원경의 해상도는 지구에서 달을 볼 때 하키 퍽을 대략적으로 추적할 수 있는 수준입니다. 하지만 EHT는 독특한 간섭계이기도 합니다. 우선, 이 장비가 생성하는 데이터가 매우 희박합니다.
이 배열은 지구상의 8개의 지리적 위치에서 각각 고유한 망원경으로만 구성됩니다. 또한 EHT는 일반적인 간섭계보다 훨씬 더 높은 주파수로 관측합니다.
따라서 소스 모델이 복잡할 수 있기 때문에 보정된 데이터를 직접 제공하기가 어려운 경우가 많습니다. 이는 신호를 손상시킬 수 있는 *이득*이라고 하는 큰 기기 효과가 있을 수 있음을 의미합니다. 이를 처리하는 한 가지 방법은 이득과 무관한 수량을 맞추는 것입니다. 이를 종종 **폐쇄 수량**이라고 합니다. 클로저 양의 유형은 [VLBI 이미징 문제 소개](@참조)에 간략하게 설명되어 있습니다.

이 튜토리얼에서는 M87의 예비 이미지를 생성하기 위해 M87의 클로저 전용 모델링을 수행합니다.

In [64]:
pwd()

"/Users/younghokim/astro/prj4_eht"

In [1]:
using Pkg

In [ ]:
# Pkg.activate("./Comrade_jl/examples/ClosureImaging")

In [ ]:
# readdir("./Comrade_jl/examples/ClosureImaging")

In [ ]:
# Pkg.instantiate()

In [ ]:
# Pkg.status("Comrade")

In [2]:
using Comrade

In [3]:
using Pyehtim

In [4]:
using StableRNGs
rng = StableRNG(123)

StableRNGs.LehmerRNG(state=0x000000000000000000000000000000f7)

In [5]:
obs = ehtim.obsdata.load_uvfits("Comrade_jl/examples/Data/SR1_M87_2017_096_lo_hops_netcal_StokesI.uvfits")

Python: <ehtim.obsdata.Obsdata object at 0x15d189ea0>

- `gain phases`가 일관성을 갖도록 데이터를 사전 처리한 후 데이터의 평균을 스캔합니다.
- 1%의 비폐쇄 오류를 유발하는 보정 문제를 해결하기 위해 1%의 시스템 노이즈를 추가합니다.

In [6]:
obs = scan_average(obs).add_fractional_noise(0.02)

[ Info: Before homogenizing we have 25 unique times
[ Info: After homogenizing we have 25 unique times


Python: <ehtim.obsdata.Obsdata object at 0x2b30e56f0>

이제 EHT 데이터 세트에서 closurer quantites를 추출합니다.

In [7]:
dlcamp, dcphase = extract_table(obs, LogClosureAmplitudes(;snrcut = 3), ClosurePhases(;snrcut = 3))

/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/younghokim/.julia/environments/v1.9/.CondaPkg/env/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(EHTObservation{Float64,Comrade.EHTLogClosureAmplitudeDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 128
, EHTObservation{Float64,Comrade.EHTClosurePhaseDatum{Float64}, ...}
  source: M87
  mjd: 57849
  frequency: 2.27070703125e11
  bandwidth: 1.856e9
  stations: [:AA, :AP, :AZ, :JC, :LM, :PV, :SM]
  nsamples: 152
)

이 모델에서는 점 소스의 래스터로 구성된 이미지 모델을 사용하고, 일부 펄스 또는 커널로 컨볼루션하여 연속 이미지 객체를 만드는 일반 이미지 모델을 사용할 것입니다.

## model/posteriror 빌드
우리 모델에서는 점 소스의 래스터로 구성된 이미지 모델을 사용하고, 일부 펄스 또는 커널로 컨볼루션하여 `Comrade.` 일반 이미지 모델과 함께 `ContinuousImage` 객체를 만들 것입니다. 이를 위해 두 개의 인자 함수를 정의하겠습니다. 첫 번째 인수는 일반적으로 모델 매개변수가 포함된 명명된 튜플입니다. 두 번째 인수는 일반적으로 상수인 모델에 대한 메타데이터를 정의합니다. 이를 통해 아래에서 정의할 이미지 `cache` 객체와 같이 상수인 인수를 모델에 명시적으로 전달할 수 있습니다.

In [8]:
function sky(θ, metadata)
    (;fg, c, σimg) = θ
    (;K, meanpr, cache) = metadata
    
    ## Construct the image model we fix the flux to 0.6 Jy in this case
    cp = meanpr .+ σimg.*c.params
    rast = ((1-fg))*K(to_simplex(AdditiveLR(), cp))
    m = ContinuousImage(rast, cache)
    
    ## Add a large-scale gaussian to deal with the over-resolved mas flux
    g = modify(Gaussian(), Stretch(μas2rad(250.0), μas2rad(250.0)), Renormalize(fg))
    return m + g
end

sky (generic function with 1 method)

이제 이미지 모델을 설정해 보겠습니다. EHT의 공칭 해상도는 20-25μas입니다. 또한 EHT는 더 큰 시야각에 그다지 민감하지 않으며, 일반적으로 60-80μas이면 M87의 콤팩트 플럭스를 설명하기에 충분합니다. 이 점을 고려하면 이미지를 설명하는 데 적은 수의 픽셀만 사용하면 됩니다.

In [9]:
npix = 32
fovxy = μas2rad(150.0)

7.27220521664304e-10

이미지 모델을 정의하려면 사용할 그리드와 사용할 FT 알고리즘(이 경우 가장 효율적인 NFFT)을 모두 지정해야 합니다.

In [12]:
result_nfft = NFFTAlg(dlcamp)

VLBISkyModels.ObservedNUFT{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Matrix{Float64}}(NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000), [-4.405690154666661e9 787577.6145833326 … -5.999801315555549e9 -15551.297851562484; -4.523017159111106e9 -1.6838098888888871e6 … 3.059254300444441e9 118294.64453124987])

In [18]:
grid = imagepixels(fovxy, fovxy, npix, npix)
buffer = IntensityMap(zeros(npix,npix), grid)
cache = create_cache(result_nfft.alg, buffer, BSplinePulse{3}())

VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}(NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}(1, 4, 2.0, :kaiser_bessel, AbstractNFFTs.POLYNOMIAL, true, false, true, 0x00000000), nothing, nothing, BSplinePulse{3}(), [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

이제 이미지를 미리 지정해야 합니다. 이 작업에서는 가우시안 마르코프 랜덤 필드를 사용하겠습니다.

앞서 가우스 마르코프 랜덤 필드를 사용하고 있으므로 먼저 `평균`을 지정해야 합니다. 이미지를 지정해야 합니다. 이 작업에서는 FWHM이 50μas인 대칭 가우시안 이미지를 사용하겠습니다.

In [19]:
using VLBIImagePriors, Distributions, DistributionsAD

fwhmfac = 2*sqrt(2*log(2))
mpr = modify(Gaussian(), Stretch(μas2rad(50.0)./fwhmfac))
imgpr = intensitymap(mpr, grid)

2-dimensional KeyedArray(NamedDimsArray(...)) with keys:
↓   X ∈ 32-element LinRange{Float64,...}
→   Y ∈ 32-element LinRange{Float64,...}
And data, 32×32 NamedDimsArray(::Matrix{Float64}, (:X, :Y)):
                 (-3.52247e-10)  …  (3.29522e-10)  (3.52247e-10)
 (-3.52247e-10)     6.37537e-8        1.32434e-7     6.37537e-8
 (-3.29522e-10)     1.32434e-7        2.751e-7       1.32434e-7
 (-3.06796e-10)     2.62014e-7        5.44273e-7     2.62014e-7
 (-2.84071e-10)     4.93724e-7        1.0256e-6      4.93724e-7
 (-2.61345e-10)     8.86092e-7   …    1.84065e-6     8.86092e-7
 (-2.38619e-10)     1.51463e-6        3.14629e-6     1.51463e-6
 (-2.15894e-10)     2.46586e-6        5.12225e-6     2.46586e-6
 (-1.93168e-10)     3.82352e-6        7.94248e-6     3.82352e-6
 (-1.70442e-10)     5.64668e-6        1.17297e-5     5.64668e-6
    ⋮                            ⋱    ⋮            
  (1.47717e-10)     7.94248e-6        1.64987e-5     7.94248e-6
  (1.70442e-10)     5.64668e-6        1.172

이제 실제로 심플렉스에서 이미지를 모델링하고 있으므로 다음을 확인해야 합니다. 변환하기 전에 평균 이미지에 단위 플럭스가 있는지 확인해야 합니다.

In [20]:
imgpr ./= flux(imgpr)
meanpr = to_real(AdditiveLR(), Comrade.baseimage(imgpr))

32×32 Matrix{Float64}:
 0.0       0.731054  1.41337  2.04695  …  1.41337  0.731054  0.0
 0.731054  1.46211   2.14442  2.778       2.14442  1.46211   0.731054
 1.41337   2.14442   2.82674  3.46032     2.82674  2.14442   1.41337
 2.04695   2.778     3.46032  4.0939      3.46032  2.778     2.04695
 2.63179   3.36285   4.04516  4.67874     4.04516  3.36285   2.63179
 3.1679    3.89895   4.58127  5.21485  …  4.58127  3.89895   3.1679
 3.65527   4.38632   5.06864  5.70222     5.06864  4.38632   3.65527
 4.0939    4.82495   5.50727  6.14085     5.50727  4.82495   4.0939
 4.4838    5.21485   5.89717  6.53075     5.89717  5.21485   4.4838
 4.82495   5.55601   6.23832  6.8719      6.23832  5.55601   4.82495
 5.11738   5.84843   6.53075  7.16433  …  6.53075  5.84843   5.11738
 5.36106   6.09211   6.77443  7.40801     6.77443  6.09211   5.36106
 5.55601   6.28706   6.96938  7.60296     6.96938  6.28706   5.55601
 ⋮                                     ⋱           ⋮         
 5.36106   6.09211   6.7

In [21]:
skymeta = (;meanpr,K=CenterImage(imgpr), cache)

(meanpr = [0.0 0.731053666996818 … 0.731053666996818 0.0; 0.731053666996818 1.4621073339936341 … 1.4621073339936341 0.731053666996818; … ; 0.731053666996818 1.4621073339936341 … 1.4621073339936341 0.731053666996818; 0.0 0.731053666996818 … 0.731053666996818 0.0], K = CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}([0.9944957386363638 -0.005326704545454383 … 0.00532670454545453 0.005504261363636347; -0.005326704545454383 0.9948393969941346 … 0.0051606030058651 0.005326704545454565; … ; 0.00532670454545453 0.0051606030058651 … 0.9948393969941348 -0.005326704545454544; 0.005504261363636347 0.005326704545454565 … -0.005326704545454544 0.9944957386363638], (32, 32)), cache = VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}(NFFTAlg{Flo

또한 이미지의 해상도에 대한 합리적인 추측이 필요합니다. 전파 천문학의 경우 이미지에서 가장 긴 기준선이 대략적으로 주어집니다. 이것을 픽셀 공간에 넣으려면 를 픽셀 공간에 넣으려면 픽셀 크기로 나눕니다.

In [22]:
beam = beamsize(dlcamp)
rat = (beam/(step(grid.X)))

5.326336637737519

가우스 마르코프 랜덤 필드를 효율적으로 만들기 위해 먼저 이미지 픽셀 수에 따라 선형적으로 스케일링할 수 있는 여러 수량을 미리 계산합니다. 이렇게 하면 다음이 크게 개선됩니다. 일반적인 가우스 프로세스에서 얻을 수 있는 N^3 스케일링을 크게 개선합니다.

In [26]:
# crcache = MarkovRandomFieldCache(meanpr);

# fmap = let crcache=crcache
#     x -> GaussMarkovRandomField(x, 1.0, crcache)
# end

# 홈페이지 튜토리얼 코드에서 변경된 부분

crcache = ConditionalMarkov(Normal, grid)

ConditionalMarkov{Normal, MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}}(MarkovRandomFieldCache{SparseArrays.SparseMatrixCSC{Float64, Int64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}, Matrix{Float64}}(sparse([1, 2, 32, 33, 993, 1, 2, 3, 34, 994  …  31, 991, 1022, 1023, 1024, 32, 992, 993, 1023, 1024], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33,

이제 마침내 이미지 이전을 형성할 수 있습니다. 이를 위해 과적합을 방지하기 위해 상관 관계 길이가 역 감마 선형에 의해 주어지는 계층적 선형을 사용합니다. 

가우스 마르코프 랜덤 필드는 매우 유연한 모델입니다.

과적합을 방지하기 위해 복잡성에 불이익을 주는 전제를 사용하는 것이 일반적입니다. 따라서 우리는 평균 이미지와의 유사성을 강화하고 평활성을 선호하는 전제 조건을 사용하고자 합니다.

In [24]:
cprior = HierarchicalPrior(crcache, InverseGamma(1.0, -log(0.01*rat)))

prior = NamedDist(c = cprior, σimg = truncated(Normal(0.0, 0.1); lower = 0.0), fg=Uniform(0.0, 1.0))

lklhd = RadioLikelihood(sky, dlcamp, dcphase;
                        skymeta = skymeta)
post = Posterior(lklhd, prior)

Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{NTuple{4, Symbol}}}}, Int64}, Comrade.Clo

## Reconstructing the Image

이 posterior에서 샘플링하려면 먼저 제약된 파라미터 공간에서 제약되지 않은 공간으로 이동하는 것이 편리합니다(즉, 변환된 posterior의 지지대가 (-∞, ∞) 인 경우). 이것은 `asflat` 함수를 사용하여 수행됩니다.

In [25]:
tpost = asflat(post)

Comrade.TransformedPosterior{Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{NTuple{4, Sy

In [34]:
tpost.transform

[1:1027] NamedTuple of transformations
  [1:1025] :c → NamedTuple of transformations
    [1:1024] :params → 32×32×
      asℝ
    [1025:1025] :hyperparams → as(Real, 0.0, ∞)
  [1026:1026] :σimg → as(Real, 0.0, ∞)
  [1027:1027] :fg → as(Real, 0.0, 1.0)

이제 후방의 치수 또는 샘플링할 매개변수의 개수를 찾을 수도 있습니다.
 ## !!! 경고
이것은 종종 예상과 다를 수 있습니다. 특히 각도 변수를 사용할 때 샘플링을 더 쉽게 하기 위해 매개변수 공간의 크기를 인위적으로 늘리는 경우가 많습니다.

In [28]:
ndim = dimension(tpost)

1027

In [77]:
typeof(tpost)

Comrade.TransformedPosterior{Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{NTuple{4, Sy

In [36]:
using ComradeOptimization
using OptimizationOptimJL
using Zygote

In [83]:
f = OptimizationFunction(tpost);

In [84]:
prob = Optimization.OptimizationProblem(f, prior_sample(rng, tpost), nothing)

OptimizationProblem. In-place: true
u0: 1027-element Vector{Float64}:
 -20.327748169760557
 -20.897191146799912
 -21.34745153451917
 -21.044382598835895
 -20.569727553243627
 -20.942074201500027
 -20.94303429031548
 -21.083833369142116
 -21.08912079048011
 -21.944611090309433
 -22.065769930326816
 -22.315669282015612
 -22.48387729083769
   ⋮
 -20.35893573741259
 -20.26173078572129
 -21.34402905296524
 -21.784602530733302
 -21.150649423501882
 -20.85253284494626
 -19.607562078817782
 -20.099516886964775
 -20.586317279323147
   6.5011342615740375
  -3.0076960067639162
  -1.5366044538576265

In [85]:
typeof(prob)

OptimizationProblem{true, OptimizationFunction{true, SciMLBase.NoAD, ComradeOptimization.var"#ℓ#3"{Comrade.TransformedPosterior{Posterior{RadioLikelihood{Comrade.ModelMetadata{typeof(sky), NamedTuple{(:meanpr, :K, :cache), Tuple{Matrix{Float64}, CenterImage{Matrix{Float64}, Tuple{Int64, Int64}}, VLBISkyModels.NUFTCache{NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, Nothing, Nothing, BSplinePulse{3}, KeyedArray{Float64, 2, NamedDimsArray{(:X, :Y), Float64, 2, Matrix{Float64}}, GriddedKeys{(:X, :Y), Tuple{LinRange{Float64, Int64}, LinRange{Float64, Int64}}, ComradeBase.NoHeader, Float64}}}}}}, Nothing, Tuple{Comrade.EHTObservation{Float64, Comrade.EHTLogClosureAmplitudeDatum{Float64}, StructArrays.StructVector{Comrade.EHTLogClosureAmplitudeDatum{Float64}, NamedTuple{(:measurement, :error, :U1, :V1, :U2, :V2, :U3, :V3, :U4, :V4, :T, :F, :quadrangle), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vecto

In [74]:
typeof(prob.u0)

Vector{Float64} (alias for Array{Float64, 1})

In [86]:
sol = solve(prob, Optim.LBFGS(); maxiters=5_00);

LoadError: Use OptimizationFunction to pass the derivatives or automatically generate them with one of the autodiff backends

홈페이지 튜토리얼 코드 동일

--> 하지만 no method matching create_cache(::NFFTAlg{Float64, AbstractNFFTs.PrecomputeFlags, UInt32}, ... 에러가 발생 

----
```julia
using ComradeOptimization
using OptimizationOptimJL
using Zygote
f = OptimizationFunction(tpost, Optimization.AutoZygote())
prob = Optimization.OptimizationProblem(f, prior_sample(rng, tpost), nothing)
sol = solve(prob, LBFGS(); maxiters=5_00);
```
----

In [57]:

# Before we analyze our solution we first need to transform back to parameter space.
xopt = transform(tpost, sol)

LoadError: UndefVarError: `sol` not defined

```julia
# First we will evaluate our fit by plotting the residuals
using Plots
residual(skymodel(post, xopt), dlcamp, ylabel="Log Closure Amplitude Res.")
# and now closure phases
#-
residual(skymodel(post, xopt), dcphase, ylabel="|Closure Phase Res.|")

# Now let's plot the MAP estimate.
import CairoMakie as CM
img = intensitymap(skymodel(post, xopt), μas2rad(150.0), μas2rad(150.0), 100, 100)
imageviz(img)

# To sample from the posterior we will use HMC and more specifically the NUTS algorithm. For information about NUTS
# see Michael Betancourt's [notes](https://arxiv.org/abs/1701.02434).
# !!! note
#     For our `metric` we use a diagonal matrix due to easier tuning.
#-
using ComradeAHMC
using Zygote
metric = DiagEuclideanMetric(ndim)
chain, stats = sample(post, AHMC(;metric, autodiff=Val(:Zygote)), 700; nadapts=500, initial_params=xopt)


# !!! warning
#     This should be run for longer!
#-
# Now that we have our posterior, we can assess which parts of the image are strongly inferred by the
# data. This is rather unique to `Comrade` where more traditional imaging algorithms like CLEAN and RML are inherently
# unable to assess uncertainty in their reconstructions.
#
# To explore our posterior let's first create images from a bunch of draws from the posterior
msamples = skymodel.(Ref(post), chain[501:2:end]);

# The mean image is then given by
using StatsBase
imgs = intensitymap.(msamples, μas2rad(150.0), μas2rad(150.0), 128, 128)
mimg = mean(imgs)
simg = std(imgs)
fig = CM.Figure(;resolution=(400, 400));
CM.image(fig[1,1], mimg,
                   axis=(xreversed=true, aspect=1, title="Mean Image"),
                   colormap=:afmhot)
CM.image(fig[1,2], simg./(max.(mimg, 1e-5)),
                   axis=(xreversed=true, aspect=1, title="1/SNR",), colorrange=(0.0, 2.0),
                   colormap=:afmhot)
CM.image(fig[2,1], imgs[1],
                   axis=(xreversed=true, aspect=1,title="Draw 1"),
                   colormap=:afmhot)
CM.image(fig[2,2], imgs[end],
                   axis=(xreversed=true, aspect=1,title="Draw 2"),
                   colormap=:afmhot)
fig

# Now let's see whether our residuals look better.
p = Plots.plot();
for s in sample(chain[501:end], 10)
    residual!(p, vlbimodel(post, s), dlcamp)
end
Plots.ylabel!("Log-Closure Amplitude Res.");
p
#-


p = Plots.plot();
for s in sample(chain[501:end], 10)
    residual!(p, vlbimodel(post, s), dcphase)
end
Plots.ylabel!("|Closure Phase Res.|");
p



# And viola, you have a quick and preliminary image of M87 fitting only closure products.
# For a publication-level version we would recommend
#    1. Running the chain longer and multiple times to properly assess things like ESS and R̂ (see [Geometric Modeling of EHT Data](@ref))
#    2. Fitting gains. Typically gain amplitudes are good to 10-20% for the EHT not the infinite uncertainty closures implicitly assume

```